Intern code:

- 0 : out of grid and wall (x and #)
- 1 : small ball
- 2 : medium ball
- 3 : small ball on top of a medium ball
- 4 : large ball
- 5 : small ball on top of a large ball
- 6 : medium ball on top of a large ball
- 7 : small ball on top of a medium ball on top of a large ball
- 8 : grass (,)
- 9 : snow (.)
- 10: character with snow on the floor (p)
- 11: character with grass on the floor (q)

In [43]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import glob
import random
import time

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

from sklearn.preprocessing import OneHotEncoder



In [44]:
###### PArametres

#mida del tauler, els bordes han de ser pared
n=16 
m=16

#recompenses
error=-100
apilar=20
bolamesgran=50
boom=500
passeig=-5



n_iterations = 2500
n_episodes_per_update = 25
n_max_steps = 50
discount_rate = 0.50


optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.categorical_crossentropy




path="C:/Users/josep/Snowman/Dades/facils/"

files=glob.glob(path+"*.txt") 


In [4]:
ree="x#,.pq1234567'" # pared,borde,herba,neu,cursor sobre neu, cursor sobre herba
                    # bola petita,bola mitjana,bola petita sobre mitjana, bola gran
                    # bola petita sobre gram, bola mitjana sobre gran, tres boles una cim altre
reem=[0,0,8,9,10,11,1,2,3,4,5,6,7,8]
def llegir_tauler(nom='C:/Users/josep/Snowman/Dades/alice.txt',n=n,m=m):
    f = open(nom, "r")
    tauler=np.zeros((n,m))
    for row,linea in enumerate(f):
        linea=linea.rstrip('\n\r\t')
        for column,car in enumerate(linea):
            res = ree.find(car)
            tauler[row,column]=reem[res]
    f.close()
    return tauler

def limits_tauler(t):
    maxx=n-1
    x=n-1
    while maxx==n-1:
        x=x-1
        trobat=False
        y=1
        while not trobat and y<m-1:
            if t[x,y]!=0:
                trobat=True
                maxx=x
            y=y+1

    maxy=m-1    
    y=m-1
    while maxy==m-1:
        y=y-1
        trobat=False
        x=1
        while not trobat and x<n-1:
            if t[x,y]!=0:
                trobat=True
                maxy=y
            x=x+1
    minx=0
    x=0
    while minx==0:
        x=x+1
        trobat=False
        y=1
        while not trobat and y<m-1:
            if t[x,y]!=0:
                trobat=True
                minx=x
            y=y+1

    miny=0
    y=0
    while miny==0:
        y=y+1
        trobat=False
        x=1
        while not trobat and x<n-1:
            if t[x,y]!=0:
                trobat=True
                miny=y
            x=x+1

    return minx,maxx,miny,maxy

def escriure_tauler(t,n=n,m=m):
    minx,maxx,miny,maxy=limits_tauler(t)
    for i in range(minx,maxx+1):
        a=""
        for j in range(miny,maxy+1):
            a=a+f"{int(t[i][j]):2d} "
        print(a)




In [6]:

encx = OneHotEncoder(sparse=False)
ency = OneHotEncoder(sparse=False)
encx.fit([[e] for e in range(n)])
ency.fit([[e] for e in range(m)])


OneHotEncoder(sparse=False)

In [37]:
inc=[[0,1,0,-1],[1,0,-1,0]]

def quants(t,num):
    res=0
    if num==1:
        for i in {1,3,5,7}:
            res += np.count_nonzero(t==i)
    elif num==2:
        for i in {2,3,6,7}:
            res += np.count_nonzero(t==i)
    elif num==4:
        for i in {4,6,7}:
            res += np.count_nonzero(t==i)
    return res



def posibles_actions(t,nout,anar=True): #anar si permetem anar o nomes moure
    actions=set()
    if anar:
        conjelim={1,2,3,4,5,6,8,9}
    else:
        conjelim={1,2,3,4,5,6}
    for i in range(1,n-1):
        for j in range(1,m-1):
            if t[i][j] in conjelim and nout[i][j]!=0: #no pot saltar a pared,bola p sobre m, bola p sobre g, bola m sobre g
                                                      #tres boles,on esta
                                                      # Si pot arribar desde on som
                    actions.add((i,j))
    #print('acgions dintre',actions)

    actions2=actions.copy()
    for (i,j) in actions2:
        if t[i][j] in {1,2,3,4,5,6}:  #Apretar, nomes podem apretar una bola sola o fer baixar
            pos=False
            e=0
            while e<4 and not pos:
                ant=t[i-inc[0][e]][j-inc[1][e]]
                if ant==10 or ant==11:
                    seg=t[i+inc[0][e]][j+inc[1][e]]
                    if t[i][j] in {1,3,5}:
                        if seg in {2,4,6,8,11} or (seg in {9,10} and (quants(t,2)+quants(t,4)<2)):
                            pos=True
                    elif t[i][j] in {2,6}:
                        if seg in {4,8,11} or (seg in {9,10} and (quants(t,4)<1)):
                            pos=True
                    elif t[i][j]==4:
                        if seg in {8,9,10,11}:
                            pos=True      
                e=e+1     
            if not pos:
                actions.remove((i,j)) 
        else: # anar {8 o 9}
            pos=False
            e=0
            while e<4 and not pos:
                seg=t[i+inc[0][e]][j+inc[1][e]]
                seg2=t[i+2*inc[0][e]][j+2*inc[1][e]] 
                if seg==1 and ((seg2 in {2,4,6,8,11}) or (seg2 in {9,10} and (quants(t,2)+quants(t,4)<2))):
                    pos=True
                elif seg==2 and ((seg2 in {4,8,11}) or (seg2 in {9,10} and (quants(t,4)<1))):
                    pos=True
                elif (seg==3 or seg==5) and (seg2 in {8,11} or (seg2 in {9,10} and (quants(t,2)+quants(t,4)<2))):
                    pos=True
                elif seg==6 and (seg2 in {8,11} or (seg2 in {9,10} and (quants(t,4)<1))):
                    pos=True
                elif seg==4 and (seg2 in {8,9,10,11}):
                    pos=True
                e=e+1
            if not pos:
                actions.remove((i,j))

    
    return(actions)



def taca(t,n=n,m=m):
    nou=np.zeros((n,m))
    res = np.where(t == 10)
    if np.size(res[0])==0:
        res = np.where(t == 11)
        if np.size(res[0])==0:
            return nou
    ([i],[j])=res
    l=[(i,j)]
    while len(l)>0:
        (i,j)=l.pop()
        val=nou[i][j]+1
        for e in range(4):
            noui=i+inc[0][e]
            nouj=j+inc[1][e]
            seg=t[noui][nouj]
            if seg in {1,2,3,4,5,6,8,9} and (nou[noui][nouj]==0 or nou[noui][nouj]>val):
                nou[noui][nouj]=val
                if seg in {8,9}:
                    l.append((noui,nouj))

    return nou


In [11]:
def entrada(t,tnou,pact,anar=False,n=n,m=m):
    posibles=np.zeros((n,m))
    for (i,j) in pact:
        posibles[i][j]=1
    return(t,posibles,tnou,anar)

In [42]:

def step(t,nout,pact,action,anar=True,n=n,m=m): # un pas
    snow=True
    res = np.where(t == 10)
    if np.size(res[0])==0:
        snow=False
        res = np.where(t == 11)
        if np.size(res[0])==0:
            return(t,error,True,anar)
    ([a],[b])=res
    
    (i,j)=action
    reward=-nout[i][j]
    if reward==0:
        if snow:
            t[a][b]=9
        else:
            t[a][b]=8
        if t[i][j] in {8,9}:
            if t[i][j]==9:
                t[i][j]=10
            else:
                t[i][j]=11
            return(t,error//5,True)
        else:
            return(t,error,True)
    else:
        if not action in pact:
            return(t,error,True,anar)
        else:
            #arreglo posició nova:
            if t[i][j]==9:
                t[i][j]=10
                anar=False
            elif t[i][j]==8:
                t[i][j]=11
                anar=False
            else:
                anar=True
                segx=i-(a-i)
                segy=j-(b-j)
                if t[i][j] in {1,3,5}:
                    if t[i][j]==1:
                        t[i][j]=11
                    elif t[i][j]==3:
                        reward=reward-apilar
                        t[i][j]=2
                    else:
                        reward=reward-apilar
                        t[i][j]=4
                    if t[segx][segy]==2:
                        reward=reward+apilar
                        t[segx][segy]=3
                    elif t[segx][segy]==4:
                        reward=reward+apilar
                        t[segx][segy]=5
                    elif t[segx][segy]==6:
                        reward=boom
                        t[segx][segy]=7
                    elif t[segx][segy]==8:
                        t[segx][segy]=1
                        reward=reward+passeig
                    else:
                        reward=reward+bolamesgran
                        t[segx][segy]=2
                elif t[i][j] in {2,6}:
                    if t[i][j]==2:
                        t[i][j]=11
                    else:
                        reward=reward-apilar
                        t[i][j]=4
                    if t[segx][segy]==4:
                        reward=reward+apilar
                        t[segx][segy]=6
                    elif t[segx][segy]==8:
                        t[segx][segy]=2
                        reward=reward+passeig
                    else:
                        reward=reward+bolamesgran
                        t[segx][segy]=4
                else:
                    t[i][j]=11
                    t[segx][segy]=4
                    reward=reward+passeig
            if t[i][j]==10 or t[i][j]==11: #arreglo on esta el cursor
                if snow: 
                    t[a][b]=9
                else:
                    t[a][b]=8
            
    return(t,reward,False,anar)
    

    


 9  9  9  1  9  9  9 
 9  0  0  9  0  0  9 
 9  9  9  1  9  9  9 
 9  9  0  9  0  9  9 
 9  9  9  1  9  9  9 
 0  9  9 10  9  9  0 


In [ ]:
'''
t=llegir_tauler(nom='C:/Users/josep/Snowman/Dades/willow.txt',n=n,m=m)
escriure_tauler(t)
anar=True

for i in range(0):
    print('>>>>>>>> ',i,' <<<<<<<<<<<<<<<<<') 
    nout=taca(t)
    pact=posibles_actions(t,nout,anar)
    print()
    escriure_tauler(nout)
    print()
    print(anar)
    print()
    print(f'{len(pact)}:{sorted(pact)}')
    pactl=list(pact)
    num=random.randint(0, len(pactl)-1)
    print(num)
    action=pactl[num]
    print('action=',action)
    
    t,reward,down,anar=step(t,nout,pact,action,anar)

    print()
    print('resutat')
    escriure_tauler(t)
    print(reward)
'''

In [8]:



def play_one_step(t, model, loss_fn, iteration, anar):
    with tf.GradientTape() as tape:
        accesibles=taca(t)
        actpos=posibles_actions(t,accesibles,anar)
        action_model = tf.squeeze(model(entrada(t,accesibles,actpos,anar))) #falta arreglar
    
        r=np.random.uniform()
        rati=max((1000-iteration)/1000*0.6,0.10)
        if r>rati:
            action=action_model 
        else:
            pactl=list(actpos)
            num=random.randint(0, len(actpos)-1)
            action=pactl[num]
       
        y_target = np.eye(4)[action[0]] #falta fer
        loss = tf.reduce_mean(loss_fn(y_target, left_proba)) #falta fer
    
    
    grads = tape.gradient(loss, model.trainable_variables)
    t,reward,done,anar=step(t,accesibles,actpos,action,anar)

    return t,reward, done, grads, rati, anar



In [7]:
def play_multiple_episodes(n_episodes, n_max_steps, model, loss_fn,iteration):
    all_rewards = []
    all_grads = []

    
    nom=files[iteration % len(files)]
    tauler=llegir_tauler(nom,n=n,m=m)
    for i in range(4):
        maxim=-1000000
        for episode in range(n_episodes):
            t=tauler.copy()
            passat=True

    
            reward=-1
            current_rewards = []
            current_grads = []
            for step in range(n_max_steps):
                t, reward, done, grads, rati, passat = play_one_step(t, model, loss_fn,iteration,passat)
                current_rewards.append(reward)
                current_grads.append(grads)
                if done:
                    break
            if sum(current_rewards)>maxim:
                maxim=sum(current_rewards)
            all_rewards.append(current_rewards)
            all_grads.append(current_grads)
        if i%2==0:
            tauler=np.flip(tauler,0)
        else:
            tauler=np.flip(tauler,1)
        
    return all_rewards, all_grads, maxim, nom, rati




In [8]:

def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]

In [13]:
tf.keras.backend.clear_session()
np.random.seed(13)
tf.random.set_seed(13)


model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(n,m)),
    #tf.keras.layers.Dense(128,activation='relu'),
    #tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(4,activation='softmax')
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 81)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                5248      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 5,508
Trainable params: 5,508
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
def prova(nom=None):
    if nom==None:
        nom=files[random.randint(0,len(files)-1)]
    
    tauler=llegir_tauler(nom,n=n,m=m)
    print(tauler)
    time.sleep(1)
    clear_output(wait=True)
    sum_rewards=0
    anar=True

    for i in range(30):
        accesibles=taca(t)
        actpos=posibles_actions(tauler,accesibles,anar)
        action_model = tf.squeeze(model(entrada(tauler,accesibles,actpos,anar))) #falta arreglar
    
        clear_output(wait=True)
        t,reward,done,anar=step(tauler,accesibles,actpos,action_model,anar)
        sum_rewards+=reward
        print(action_model,i,sum_rewards,done)
        print(tauler)
        time.sleep(5)
        if done:
            break
        clear_output(wait=True)

In [ ]:
for iteration in range(n_iterations):
    if (iteration+1)%100==0:
        prova()
    
    all_rewards, all_grads, maxim = play_multiple_episodes(n_episodes_per_update, n_max_steps, model, loss_fn,iteration)

    total_rewards = sum([sum(ar) for ar in all_rewards])                   # Not shown in the book
    print("\rIteration: {}, mean rewards: {:.1f} Mov: {} maxim: {}     ".format(iteration, total_rewards / n_episodes_per_update,Q,maxim), end="") # Not shown

    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
    
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
        

        
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))


In [ ]:
model.save('snownou-1.h5')